# BYTE PAIR ENCODING

Byte pair encding is used in GPT, GPT-2, BART, RoBERTA and DeBERTa. It creates a vocabulary size based on the input text. First the texts are divided into characters and merged based on the number of frequency of the merged tokens. BPE merge together characters which occur frequently

In [1]:
#Define a corpus of words
corpus = [
    "This is the beginning of Hugging Face Course",
    "This chapter is about tokenization",
    "This section shows several tokenization algorithm",
    "Hopefully you will be able to understand how they are trained to generate tokens"
]

In [2]:
#Define a pre-tokenizer. Here i have used gpt2 since is uses BPE for tokenization
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [3]:
#Compute the frequencies of each word
from collections import defaultdict

word_freqs = defaultdict(int)
for text in corpus:
  words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
  new_words = [word for word, offset in words_with_offsets]
  for word in new_words:
    word_freqs[word]+=1

In [4]:
print(word_freqs)

defaultdict(<class 'int'>, {'This': 3, 'Ġis': 2, 'Ġthe': 1, 'Ġbeginning': 1, 'Ġof': 1, 'ĠHugging': 1, 'ĠFace': 1, 'ĠCourse': 1, 'Ġchapter': 1, 'Ġabout': 1, 'Ġtokenization': 2, 'Ġsection': 1, 'Ġshows': 1, 'Ġseveral': 1, 'Ġalgorithm': 1, 'Hopefully': 1, 'Ġyou': 1, 'Ġwill': 1, 'Ġbe': 1, 'Ġable': 1, 'Ġto': 2, 'Ġunderstand': 1, 'Ġhow': 1, 'Ġthey': 1, 'Ġare': 1, 'Ġtrained': 1, 'Ġgenerate': 1, 'Ġtokens': 1})


In [5]:
#Find the base vocabulary formed by the characters used in corpus
alphabet = []

for word in word_freqs.keys():
  for letter in word:
    if letter not in alphabet:
      alphabet.append(letter)

alphabet.sort()

print(alphabet)

['C', 'F', 'H', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ']


In [6]:
vocab = ["<|endoftext|>"] + alphabet.copy()

In [9]:
#Split each word into individual character

splits = {word: [c for c in word] for word in word_freqs.keys()}

In [33]:
splits

{'This': ['T', 'h', 'i', 's'],
 'Ġis': ['Ġ', 'i', 's'],
 'Ġthe': ['Ġ', 't', 'h', 'e'],
 'Ġbeginning': ['Ġ', 'b', 'e', 'g', 'i', 'n', 'n', 'i', 'n', 'g'],
 'Ġof': ['Ġ', 'o', 'f'],
 'ĠHugging': ['Ġ', 'H', 'u', 'g', 'g', 'i', 'n', 'g'],
 'ĠFace': ['Ġ', 'F', 'a', 'c', 'e'],
 'ĠCourse': ['Ġ', 'C', 'o', 'u', 'r', 's', 'e'],
 'Ġchapter': ['Ġ', 'c', 'h', 'a', 'p', 't', 'e', 'r'],
 'Ġabout': ['Ġ', 'a', 'b', 'o', 'u', 't'],
 'Ġtokenization': ['Ġ',
  't',
  'o',
  'k',
  'e',
  'n',
  'i',
  'z',
  'a',
  't',
  'i',
  'o',
  'n'],
 'Ġsection': ['Ġ', 's', 'e', 'c', 't', 'i', 'o', 'n'],
 'Ġshows': ['Ġ', 's', 'h', 'o', 'w', 's'],
 'Ġseveral': ['Ġ', 's', 'e', 'v', 'e', 'r', 'a', 'l'],
 'Ġalgorithm': ['Ġ', 'a', 'l', 'g', 'o', 'r', 'i', 't', 'h', 'm'],
 'Hopefully': ['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y'],
 'Ġyou': ['Ġ', 'y', 'o', 'u'],
 'Ġwill': ['Ġ', 'w', 'i', 'l', 'l'],
 'Ġbe': ['Ġ', 'b', 'e'],
 'Ġable': ['Ġ', 'a', 'b', 'l', 'e'],
 'Ġto': ['Ġ', 't', 'o'],
 'Ġunderstand': ['Ġ', 'u', 'n', 'd', 

In [15]:
def compute_pair_freqs(splits):
  pair_freqs = defaultdict(int)
  for word, freq in word_freqs.items():
    split = splits[word]
    if len(split) == 1:
      continue
    for i in range(len(split) - 1):
      pair = (split[i], split[i+1])
      pair_freqs[pair]+=freq
  return pair_freqs

In [16]:
pair_freqs = compute_pair_freqs(splits)

In [17]:
for i, key in enumerate(pair_freqs.keys()):
  print(f"{key}: {pair_freqs[key]}")
  if i>=5:
    break

('T', 'h'): 3
('h', 'i'): 3
('i', 's'): 5
('Ġ', 'i'): 2
('Ġ', 't'): 8
('t', 'h'): 3


In [29]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
  if max_freq is None or max_freq<freq:
    best_pair = pair
    max_freq = freq

print(best_pair, max_freq)

('Ġ', 't') 8


In [30]:
#So the first merge is 'Gt and we add this to vocabulary
merges = {('Ġ', 't'): "Ġt"}
vocab.append("Ġt")

In [34]:
#Define a function that merge in splits dictionary
def merge_pair(a,b,splits):
  for word in word_freqs:
    split = splits[word]
    if len(split) == 1:
      continue

    i=0
    while i<len(split) - 1:
      if split[i] == a and split[i+1] == b:
        split = split[:i] + [a+b] + split[i+2:]
      else:
        i+=1
    splits[word] = split
  return splits

In [35]:
splits = merge_pair("Ġ","t",splits)
print(splits["Ġtrained"])

['Ġt', 'r', 'a', 'i', 'n', 'e', 'd']
